# HW 2: Deep Bag-of-Words

![words.jpeg](https://cdn-images-1.medium.com/max/1600/0*JpqZhCNsQ_OGaRkB.jpg)

<br>

In this homework, you will be implementing a deep averaging network, detailed in [Deep Unordered Composition  Rivals Syntactic Methods for Text Classification by Iyyer et al. (2015)](https://people.cs.umass.edu/~miyyer/pubs/2015_acl_dan.pdf) and training it to do sentiment analysis on the Stanford Sentiment Treebank.

<br>

**Please use all of the starter code that is provided, do not make any changes to the data processing, evaluation, and training functions. Only add code were you're asked to.**

<br>

# Part 0: Read Paper!

Read [Deep Unordered Composition  Rivals Syntactic Methods for Text Classification by Iyyer et al. (2015)](https://people.cs.umass.edu/~miyyer/pubs/2015_acl_dan.pdf).

## Data Processing

Make sure you've downloaded the Stanford Sentiment Treebank that was used in lab. You can find it [here](http://nlp.stanford.edu/sentiment/trainDevTestTrees_PTB.zip).

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import re
import random

random.seed(1)
sst_home = 'trees'

# Let's do 2-way positive/negative classification instead of 5-way
easy_label_map = {0:0, 1:0, 2:None, 3:1, 4:1}
    # so labels of 0 and 1 in te 5-wayclassificaiton are 0 in the 2-way. 3 and 4 are 1, and 2 is none
    # because we don't have a neautral class. 

PADDING = "<PAD>"
UNKNOWN = "<UNK>"
max_seq_length = 20

def load_sst_data(path):
    data = []
    with open(path) as f:
        for i, line in enumerate(f): 
            example = {}
            example['label'] = easy_label_map[int(line[1])]
            if example['label'] is None:
                continue
            
            # Strip out the parse information and the phrase labels---we don't need those here
            text = re.sub(r'\s*(\(\d)|(\))\s*', '', line)
            example['text'] = text[1:]
            data.append(example)

    random.seed(1)
    random.shuffle(data)
    return data
     
training_set = load_sst_data(sst_home + '/train.txt')
dev_set = load_sst_data(sst_home + '/dev.txt')
test_set = load_sst_data(sst_home + '/test.txt')

<br>

Next, we'll extract the vocabulary from the data, index each token, and finally convert the sentences into lists of indexed tokens. We are also padding and truncating all sentences to be of length=20. (Why? Think about how to handle batching. This is certainly not the only way! This is just simple.)

In [6]:
import collections
import numpy as np

def tokenize(string):
    return string.split()

def build_dictionary(training_datasets):
    """
    Extract vocabulary and build dictionary.
    """  
    word_counter = collections.Counter()
    for i, dataset in enumerate(training_datasets):
        for example in dataset:
            word_counter.update(tokenize(example['text']))
        
    vocabulary = set([word for word in word_counter])
    vocabulary = list(vocabulary)
    vocabulary = [PADDING, UNKNOWN] + vocabulary
        
    word_indices = dict(zip(vocabulary, range(len(vocabulary))))

    return word_indices, len(vocabulary)

def sentences_to_padded_index_sequences(word_indices, datasets):
    """
    Annotate datasets with feature vectors. Adding right-sided padding. 
    """
    for i, dataset in enumerate(datasets):
        for example in dataset:
            example['text_index_sequence'] = torch.zeros(max_seq_length)

            token_sequence = tokenize(example['text'])
            padding = max_seq_length - len(token_sequence)

            for i in range(max_seq_length):
                if i >= len(token_sequence):
                    index = word_indices[PADDING]
                    pass
                else:
                    if token_sequence[i] in word_indices:
                        index = word_indices[token_sequence[i]]
                    else:
                        index = word_indices[UNKNOWN]
                example['text_index_sequence'][i] = index

            example['text_index_sequence'] = example['text_index_sequence'].long().view(1,-1)
            example['label'] = torch.LongTensor([example['label']])


word_to_ix, vocab_size = build_dictionary([training_set])
sentences_to_padded_index_sequences(word_to_ix, [training_set, dev_set, test_set])

In [65]:
print("Size of training dataset:", len(training_set))
print("\nFirst padded and indexified example in training data:\n", training_set[0])
vocab_size

Size of training dataset: 6920

First padded and indexified example in training data:
 {'label': tensor([0]), 'text': 'Yet another entry in the sentimental oh-those-wacky-Brits genre that was ushered in by The Full Monty and is still straining to produce another smash hit .', 'text_index_sequence': tensor([[ 9528,  3900, 10657,  1974,   153, 11985,  6137,   947,  7656,  8279,
          8259,  1974,  7851, 13476,  2108,  8315,  9242,  4143,  8823,  5485]])}


16284

<br>
### Batichify data
We're going to be doign mini-batch training. The following code makes data iterators and a batchifying function.

In [8]:
# This is the iterator we'll use during training. 
# It's a generator that gives you one batch at a time.
def data_iter(source, batch_size):
    dataset_size = len(source)
    start = -1 * batch_size
    order = list(range(dataset_size))
    random.shuffle(order)

    while True:
        start += batch_size
        if start > dataset_size - batch_size:
            # Start another epoch.
            start = 0
            random.shuffle(order)   
        batch_indices = order[start:start + batch_size]
        batch = [source[index] for index in batch_indices]
        yield [source[index] for index in batch_indices]

# This is the iterator we use when we're evaluating our model. 
# It gives a list of batches that you can then iterate through.
def eval_iter(source, batch_size):
    batches = []
    dataset_size = len(source)
    start = -1 * batch_size
    order = list(range(dataset_size))
    random.shuffle(order)

    while start < dataset_size - batch_size:
        start += batch_size
        batch_indices = order[start:start + batch_size]
        batch = [source[index] for index in batch_indices]
        if len(batch) == batch_size:
            batches.append(batch)
        else:
            continue
        
    return batches

# The following function gives batches of vectors and labels, 
# these are the inputs to your model and loss function
def get_batch(batch):
    vectors = []
    labels = []
    for dict in batch:
        vectors.append(dict["text_index_sequence"])
        labels.append(dict["label"])
    return vectors, labels


## Evaluation

We'll be looking at accuracy as our evaluation metric.

In [9]:
# This function outputs the accuracy on the dataset, we will use it during training.
def evaluate(model, data_iter):
    model.eval()
    correct = 0
    total = 0
    for i in range(len(data_iter)):
        vectors, labels = get_batch(data_iter[i])
        vectors = torch.stack(vectors).squeeze()
        labels = torch.stack(labels).squeeze()

        output = model(vectors)
        
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
      
    return correct / float(total)

<br>

## Training Loop

The following function trains the model and reports model accuracy on the train and dev set every 5 epochs.

In [252]:
def training_loop(batch_size, num_epochs, model, loss_, optim, training_iter, dev_iter,
                  train_eval_iter, verbose=True):
    step = 0
    epoch = 0
    total_batches = int(len(training_set) / batch_size)
    accuracies = []
    while epoch <= num_epochs:
        model.train()
        vectors, labels = get_batch(next(training_iter)) 
        vectors = torch.stack(vectors).squeeze() # batch_size, seq_len
        labels = torch.stack(labels).squeeze()
        
    
        model.zero_grad()
        
        output = model(vectors)

        lossy = loss_(output, labels)
        lossy.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optim.step()
        

        if step % total_batches == 0:
            model.eval()
            if epoch % 5 == 0:
                train_acc = evaluate(model, train_eval_iter)
                eval_acc = evaluate(model, dev_iter)
                accuracies.append(eval_acc)
                if verbose:
                    print("Epoch %i; Step %i; Loss %f; Train acc: %f; Dev acc %f" 
                          %(epoch, step, lossy.item() ,\
                            train_acc, eval_acc))
            epoch += 1
        step += 1
    
    best_dev = max(accuracies)
    print("Best dev accuracy is {}".format(best_dev))
    return best_dev

<br>

# Part 1: Implement DAN (40 points)

Following the [paper](https://people.cs.umass.edu/~miyyer/pubs/2015_acl_dan.pdf), implement the Deep Averaging Network (DAN).

Implementation details,
- Instead of using \code{tanh} activations however, use \code{ReLU}. 
- Make the number of layers a variable, not a fixed value.
- Make sure to implement word-dropout.

In [349]:
class DAN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size, 
                 batch_size, n_layers, drop_rate):
        super(DAN, self).__init__()
        
        self.embed = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embedding_size = embedding_dim
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_size = batch_size
        self.n_layers = n_layers
        self.drop_rate = drop_rate
        self.activation = nn.ReLU()
        
        """
        YOUR CODE GOES HERE
        
        """
        self.layer_0 = nn.Linear(embedding_dim,hidden_size)
        

        for i in range(hidden_size-1):
            setattr(self, 'l{}'.format(i), nn.Linear(hidden_size,hidden_size))
        
            
        self.decoder = nn.Linear(hidden_size,output_size)
        self.init_weights()
    
    def drop_out(self,x):
            mask = np.random.binomial(1,self.drop_rate, x.shape[1])
            # generate bernoulli with number of words in each sentence
            x_dropout = torch.tensor(mask[:, np.newaxis]).float() * x
            return x_dropout
        
    def forward(self,x):
        """
        YOUR CODE GOES HERE
        """
        output = torch.zeros((self.batch_size, self.output_size))
        requires_grad=True
        
        x_emb = self.embed(x)  

        # make sure dropout only go through train_iter, not eval_iter
        if self.training:
            x_emb = self.drop_out(x_emb)
        
        a_v = torch.mean(x_emb,dim=1)

        hidden = self.activation(self.layer_0(a_v))
       
        for i in range(1, self.n_layers):
            func = getattr(self, 'l{}'.format(i))
            hidden = torch.mm(hidden, func.weight)+func.bias
            hidden = self.activation(hidden)
            
        
        output = self.decoder(hidden)
        
        return output

    def init_hidden(self):
        h0 = torch.zeros(self.batch_size, self.hidden_size)
        
        return h0
    
    def init_weights(self):
        initrange = 0.1
        lin_layers = [self.layer_0, self.decoder]
        for i in range(1, self.n_layers):
            lin_layers.append(getattr(self, 'l{}'.format(i)))
        em_layer =  [self.embed]
     
        for layer in lin_layers+em_layer:
            layer.weight.data.uniform_(-initrange, initrange)
            if layer in lin_layers:
                layer.bias.data.fill_(0)

### Train the model!

** Please use the hyperparameters and optimizer provided below. Do not make changes here. **

In [350]:
# Hyperparameters 
input_size = vocab_size
num_labels = 2
hidden_dim = 24
embedding_dim = 8
batch_size = 256
num_layers = 2
learning_rate = 0.001
drop_rate = 0.4
num_epochs = 30


# Build and initialize the model
dan = DAN(vocab_size, embedding_dim, hidden_dim, num_labels, batch_size, num_layers, drop_rate)
dan.init_weights()

# Define loss function and optimizer
loss = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(dan.parameters(), lr=learning_rate)

# Build data iterators
training_iter = data_iter(training_set, batch_size)
train_eval_iter = eval_iter(training_set[0:500], batch_size)
dev_iter = eval_iter(dev_set[:500], batch_size)

# Train the model
training_loop(batch_size, num_epochs, dan, loss, optimizer, training_iter, dev_iter, train_eval_iter)

Epoch 0; Step 0; Loss 0.693146; Train acc: 0.488281; Dev acc 0.546875
Epoch 5; Step 135; Loss 0.673497; Train acc: 0.488281; Dev acc 0.546875
Epoch 10; Step 270; Loss 0.471621; Train acc: 0.847656; Dev acc 0.804688
Epoch 15; Step 405; Loss 0.268915; Train acc: 0.949219; Dev acc 0.804688
Epoch 20; Step 540; Loss 0.252062; Train acc: 0.964844; Dev acc 0.816406
Epoch 25; Step 675; Loss 0.155931; Train acc: 0.980469; Dev acc 0.816406
Epoch 30; Step 810; Loss 0.227976; Train acc: 0.984375; Dev acc 0.812500
Best dev accuracy is 0.81640625


0.81640625

<br>

# Part 2: Hyperparameter tuning (40 points)

Tune the DAN for learning rate, number of layers, and drop-out rate. Select a range for each parameter and then do a random search over these hyperparameters, trying a minimum 5 permutations of hyperparameters. Report results and the best hyperparameters you found. Do you see any patterns in your results?

In [324]:
"""
YOUR CODE GOES HERE
"""
# select a range for each parameter.
learning_rate_list = np.arange(0.0001,0.01,0.001)
drop_rate_list = np.arange(0.1,1.1,0.1)
n_layers_list = np.arange(2,22,2)

# randint generate index for each parameter group, and try 5 different combinations 
random.seed(2860)
index_learning = np.random.randint(10, size=5)
index_drop = np.random.randint(10, size=5)
index_layers = np.random.randint(10, size=5)

learning_list = learning_rate_list[index_learning]
drop_list = drop_rate_list[index_drop]
layers_list = n_layers_list[index_layers]

# iterate over all parameter group to generate a permutation
for i in range(5):
    dan = DAN(vocab_size, embedding_dim, hidden_dim, num_labels, batch_size, layers_list[i], drop_list[i])
    dan.init_weights()

    # Define loss function and optimizer
    loss = nn.CrossEntropyLoss()  
    optimizer = torch.optim.Adam(dan.parameters(), lr=learning_list[i])

    # Build data iterators
    training_iter = data_iter(training_set, batch_size)
    train_eval_iter = eval_iter(training_set[0:500], batch_size)
    dev_iter = eval_iter(dev_set[:500], batch_size)

    # Train the model
    training_loop(batch_size, num_epochs, dan, loss, optimizer, training_iter, dev_iter, train_eval_iter)

Epoch 0; Step 0; Loss 0.693146; Train acc: 0.535156; Dev acc 0.539062
Epoch 5; Step 135; Loss 0.691662; Train acc: 0.535156; Dev acc 0.539062
Epoch 10; Step 270; Loss 0.695426; Train acc: 0.535156; Dev acc 0.539062
Epoch 15; Step 405; Loss 0.693902; Train acc: 0.535156; Dev acc 0.539062
Epoch 20; Step 540; Loss 0.689629; Train acc: 0.535156; Dev acc 0.539062
Epoch 25; Step 675; Loss 0.689568; Train acc: 0.535156; Dev acc 0.539062
Epoch 30; Step 810; Loss 0.692056; Train acc: 0.535156; Dev acc 0.539062
Best dev accuracy is 0.5390625
Epoch 0; Step 0; Loss 0.693146; Train acc: 0.539062; Dev acc 0.472656
Epoch 5; Step 135; Loss 0.692445; Train acc: 0.539062; Dev acc 0.472656
Epoch 10; Step 270; Loss 0.688587; Train acc: 0.539062; Dev acc 0.472656
Epoch 15; Step 405; Loss 0.690858; Train acc: 0.539062; Dev acc 0.472656
Epoch 20; Step 540; Loss 0.690279; Train acc: 0.539062; Dev acc 0.472656
Epoch 25; Step 675; Loss 0.689154; Train acc: 0.539062; Dev acc 0.472656
Epoch 30; Step 810; Loss 0.6

In [327]:
print('the best results is: 0.8125', 'with learning rate:',learning_list[4],'drop rate:',drop_list[4],'and num layers:',layers_list[4])

the best results is: 0.8125 with learning rate: 0.0061 drop rate: 0.30000000000000004 and num layers: 4


<br>

# Part 3: short questions (20 points)

1. Name, and briefly describe, 3 other possible composition functions, instead of the DAN, you could use to build sentence representations.
2. Explain how dropout regularizes a model.
3. What are the shortcomings for training for a fixed number of epochs? Give an alternative.
4. Explain why you might use random search rather than grid search.

_Bonus (5 points): briefly describe the Nelder–Mead method and how you might use it to do hyperparamter tuning. What are the tradeoffs between using Nelder-Mead vs random search?_

1.  - RecRnn: A recurrent neural network that place softmax layers above every internal node to avoid vanishing gradient issues.
    - Tree-LSTM: Improves RNN architectures to learn long-term dependencies.
    - RecNTN: a recursive neural network which relies on external structural features given by a parse tree and perform best among the RecRNNs.
    - GRU: a gating mechanism in recurrent nerual network with forget gate that has fewer parameter than LSTM.

2. In dropout process, randomly selected neurons are ignored during training. Dropout regularizes by making the training process noisy, forcing nodes within a layer to probabilistically take more or less responsibility to the inputs. Then the neurons become less sensitive to specific weights of neurons.

3. When epochs number is too small, it will not be trained sufficiently. When the epochs number is too large, the model tends to overfit. Alternatives: early stopping(not overfitting,loss-reduced converges to a certain range, accuracy not improved after 'n' epochs, etc), regularizations(dropout), use reinforcement learning.

4. Compared to grid search, random search can solve larg-scale problems efficiently and save computational cost, since it is highly likely that the whole action space has been reached because of the randomness.

Bonus: 
- Nelder-Mead method is an optimization method that minimizes the objective function by repeating its evaluation at each vertex of the simplex and by replacing points according to reflection, expansion, contraction and shrinking procedure.
- Using Nelder-Mead method saves search space. Compared to random search, there is no injected randomness, but allows for noisy loss measurements.
- On the other side, although it practically proves to be effective, there is no general convergence theory.